In [51]:
import math
import os
from os.path import join
import time

from absl import app
from absl import flags
from absl import logging
from tqdm import tqdm
import numpy as np

from keras.optimizers import Adam

# from data import Embedding, MultiLanguageEmbedding, \
#     LazyIndexCorpus,  Word2vecIterator, BilbowaIterator

from data import *
from model import get_model, word2vec_loss, bilbowa_loss, strong_pair_loss, weak_pair_loss



In [52]:
os.getcwd()


'/Applications/Setapp/GD/research/cross-lingual/bilingual_dict_embeddings/reference/bilbowa'

In [55]:
emb0 = Embedding("./data_root/withctx.en-fr.en.50.1.txt")


100%|██████████| 995003/995003 [00:40<00:00, 24839.62 words/s]


FileNotFoundError: [Errno 2] No such file or directory: './data_root/withctx.en-fr.fr.50.1.txt '

In [56]:
emb1 = Embedding("./data_root/withctx.en-fr.fr.50.1.txt")
emb = MultiLanguageEmbedding(emb0, emb1)
vocab = emb.get_vocab()
emb_matrix = emb.get_emb()


100%|██████████| 432455/432455 [00:21<00:00, 20523.31 words/s]


In [57]:
strong, weak = read_pair()
strong_id, weak_id, l0_dict, l1_dict = pair2id(strong, weak, emb)

In [58]:
ctxemb0 = Embedding("./data_root/withctx.en-fr.en.50.1.txt.ctx")
ctxemb1 = Embedding("./data_root/withctx.en-fr.fr.50.1.txt.ctx")
ctxemb = MultiLanguageEmbedding(ctxemb0, ctxemb1)
ctxvocab = ctxemb.get_vocab()
ctxemb_matrix = ctxemb.get_emb()

100%|██████████| 432455/432455 [00:12<00:00, 33520.77 words/s]


In [59]:
assert tuple(ctxvocab) == tuple(vocab)

In [62]:
mono_max_lines = 10000
mono0 = LazyIndexCorpus("./data_root/en_mono",
        max_lines=mono_max_lines)

mono1 = LazyIndexCorpus("./data_root/en_mono",mono_max_lines)

In [64]:
multi_max_lines = 10000
multi0 = LazyIndexCorpus("./data_root/en_multi",max_lines=multi_max_lines)
multi1 = LazyIndexCorpus("./data_root/fr_multi",max_lines=multi_max_lines)

In [65]:
mono0_unigram_table = mono0.get_unigram_table(vocab_size=len(vocab))
mono1_unigram_table = mono1.get_unigram_table(vocab_size=len(vocab))

In [67]:
emb_subsample = 1e-5
word2vec_negative_size = 10
word2vec_batch_size = 100000
mono0_iterator = Word2vecIterator(
        mono0,
        mono0_unigram_table,
        subsample=emb_subsample,
        window_size=word2vec_negative_size,
        negative_samples=word2vec_negative_size,
        batch_size=word2vec_batch_size,
    )

mono1_iterator = Word2vecIterator(
        mono1,
        mono1_unigram_table,
        subsample=emb_subsample,
        window_size=word2vec_negative_size,
        negative_samples=word2vec_negative_size,
        batch_size=word2vec_batch_size,
    )

In [68]:
bilbowa_sent_length = 50
bilbowa_batch_size = 100
multi_iterator = BilbowaIterator(
    multi0,
    multi1,
    mono0_unigram_table,
    mono1_unigram_table,
    subsample=emb_subsample,
    length=bilbowa_sent_length,
    batch_size=bilbowa_batch_size,
)

In [70]:
# strong pair iterator
strong_batch_size = 1000
strong_negative_size = 10
strong_pair_iterator = strong_pairIterator(
    strong_id,
    mono0_unigram_table,
    mono1_unigram_table,
    batch_size = strong_batch_size,
    negative_samples = strong_negative_size,
    l0_dict = l0_dict,
    l1_dict = l1_dict
)

# weak pair iterator
weak_batch_size = 3000
weak_negative_size = 10
weak_pair_iterator = weak_pairIterator(
    weak_id,
    mono0_unigram_table,
    mono1_unigram_table,
    batch_size=weak_batch_size,
    negative_samples=weak_negative_size,
    l0_dict=l0_dict,
    l1_dict=l1_dict
)


In [73]:
emb_dim = 50
encoder_desc_length = 15
(
    word2vec_model,
    bilbowa_model,
    strong_pair_model,
    weak_pair_model,
    word2vec_model_infer,
    bilbowa_model_infer,
    strong_pair_model_infer,
    weak_pair_model_infer
) = get_model(
    nb_word=len(vocab),
    dim=emb_dim,
    length=bilbowa_sent_length,
    desc_length=encoder_desc_length,
    word_emb_matrix=emb_matrix,
    context_emb_matrix=ctxemb_matrix,
)


OUTPUT Tensor("dot_2/MatMul:0", shape=(?, 1, 1), dtype=float32)
OUTPUT Tensor("flatten_3/Reshape:0", shape=(?, ?), dtype=float32)
OUTPUT Tensor("multiply_1/mul:0", shape=(?, ?), dtype=float32)
